# Build a simulation using the Python API

Currently, this notbook manipulates the simulation by directly placing objects inside of the attributes of the network and domain. It should be refactored when proper methods exist for adding these objects.


Import the Simulation class

In [1]:
from primaite.simulator.sim_container import Simulation


Create an empty simulation. By default this has a network with no nodes or links, and a domain controller with no accounts.

Let's use the simulation's `describe_state()` method to verify that it is empty.

In [2]:
my_sim = Simulation()
my_sim.describe_state()

{'uuid': '5304ed6d-de4c-408c-ae24-ada32852d196',
 'network': {'uuid': 'fa17dfe8-81a1-4c7f-8c5b-8c2d3b1e8756',
  'nodes': {},
  'links': {}},
 'domain': {'uuid': '320cbb83-eb1b-4911-a4f0-fc46d8038a8a', 'accounts': {}}}

## Add nodes

In [3]:
from primaite.simulator.network.hardware.base import Node


In [4]:
my_pc = Node(hostname="primaite_pc",)
my_server = Node(hostname="google_server")

# TODO: when there is a proper function for adding nodes, use it instead of manually adding.

my_sim.network.nodes[my_pc.uuid] = my_pc
my_sim.network.nodes[my_server.uuid] = my_server


## Connect the nodes

In [5]:
from primaite.simulator.network.hardware.base import NIC, Link, Switch


In [6]:
my_swtich = Switch(hostname="switch1", num_ports=12)

pc_nic = NIC(ip_address="130.1.1.1", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_pc.connect_nic(pc_nic)


server_nic = NIC(ip_address="130.1.1.2", gateway="130.1.1.255", subnet_mask="255.255.255.0")
my_server.connect_nic(server_nic)


pc_to_switch = Link(endpoint_a=pc_nic, endpoint_b=my_swtich.switch_ports[1])
server_to_swtich = Link(endpoint_a=server_nic, endpoint_b=my_swtich.switch_ports[2])

my_sim.network.links[pc_to_switch.uuid] = pc_to_switch
my_sim.network.links[server_to_swtich.uuid] = server_to_swtich

2023-08-20 18:42:51,310: NIC 5c:b6:26:c0:86:61/130.1.1.1 connected to Link 5c:b6:26:c0:86:61/130.1.1.1<-->01:ef:b1:a3:24:72
2023-08-20 18:42:51,311: SwitchPort 01:ef:b1:a3:24:72 connected to Link 5c:b6:26:c0:86:61/130.1.1.1<-->01:ef:b1:a3:24:72
2023-08-20 18:42:51,314: NIC f6:de:1e:63:8e:7f/130.1.1.2 connected to Link f6:de:1e:63:8e:7f/130.1.1.2<-->30:9e:c8:d4:5d:f3
2023-08-20 18:42:51,315: SwitchPort 30:9e:c8:d4:5d:f3 connected to Link f6:de:1e:63:8e:7f/130.1.1.2<-->30:9e:c8:d4:5d:f3


## Add files and folders to nodes


In [7]:
from primaite.simulator.file_system.file_system_file_type import FileSystemFileType
from primaite.simulator.file_system.file_system_file import FileSystemFile

In [8]:
my_pc_downloads_folder = my_pc.file_system.create_folder("downloads")
my_pc_downloads_folder.add_file(FileSystemFile(name="firefox_installer.zip",file_type=FileSystemFileType.ZIP))

In [9]:
my_server_folder = my_server.file_system.create_folder("static")
my_server.file_system.create_file("favicon.ico", file_type=FileSystemFileType.PNG)

FileSystemFile(uuid='253e4606-0f6d-4e57-8db0-6fa7e331ecea', name='favicon.ico', size=40.0, file_type=<FileSystemFileType.PNG: '11'>, action_manager=None)

## Add applications to nodes

In [10]:
from primaite.simulator.system.applications.application import Application, ApplicationOperatingState
from primaite.simulator.system.software import SoftwareHealthState, SoftwareCriticality
from primaite.simulator.network.transmission.transport_layer import Port

# no applications exist yet so we will create our own.
class MSPaint(Application):
    def describe_state(self):
        return super().describe_state()

In [11]:
mspaint = MSPaint(name = "mspaint", health_state_actual=SoftwareHealthState.GOOD, health_state_visible=SoftwareHealthState.GOOD, criticality=SoftwareCriticality.MEDIUM, ports={Port.HTTP}, operating_state=ApplicationOperatingState.RUNNING,execution_control_status='manual')

In [12]:
my_pc.applications[mspaint.uuid] = mspaint

## Create a domain account

In [13]:
from primaite.simulator.domain.account import Account, AccountType


In [14]:
acct = Account(username="admin", password="admin12", account_type=AccountType.USER)
my_sim.domain.accounts[acct.uuid] = acct

Verify that the state dictionary contains no non-serialisable objects.

In [15]:
my_sim.describe_state()

{'uuid': '5304ed6d-de4c-408c-ae24-ada32852d196',
 'network': {'uuid': 'fa17dfe8-81a1-4c7f-8c5b-8c2d3b1e8756',
  'nodes': {'1fa46446-6681-4e25-a3ba-c4c2cc564630': {'uuid': '1fa46446-6681-4e25-a3ba-c4c2cc564630',
    'hostname': 'primaite_pc',
    'operating_state': 0,
    'NICs': {'09ca02eb-7733-492c-9eff-f0d6b6ebeeda': {'uuid': '09ca02eb-7733-492c-9eff-f0d6b6ebeeda',
      'ip_adress': '130.1.1.1',
      'subnet_mask': '255.255.255.0',
      'gateway': '130.1.1.255',
      'mac_address': '5c:b6:26:c0:86:61',
      'speed': 100,
      'mtu': 1500,
      'wake_on_lan': False,
      'dns_servers': [],
      'enabled': False}},
    'file_system': {'uuid': '8b533e31-04e9-4838-839d-0656ace3e57a',
     'folders': {'b450c223-872c-4fe0-90cc-9da80973eaad': {'uuid': 'b450c223-872c-4fe0-90cc-9da80973eaad',
       'name': 'downloads',
       'size': 1000.0,
       'files': {'8160e685-a76f-4171-8a12-3d6b32a9ea16': {'uuid': '8160e685-a76f-4171-8a12-3d6b32a9ea16',
         'name': 'firefox_installer.z

In [16]:
import json
json.dumps(my_sim.describe_state())

'{"uuid": "5304ed6d-de4c-408c-ae24-ada32852d196", "network": {"uuid": "fa17dfe8-81a1-4c7f-8c5b-8c2d3b1e8756", "nodes": {"1fa46446-6681-4e25-a3ba-c4c2cc564630": {"uuid": "1fa46446-6681-4e25-a3ba-c4c2cc564630", "hostname": "primaite_pc", "operating_state": 0, "NICs": {"09ca02eb-7733-492c-9eff-f0d6b6ebeeda": {"uuid": "09ca02eb-7733-492c-9eff-f0d6b6ebeeda", "ip_adress": "130.1.1.1", "subnet_mask": "255.255.255.0", "gateway": "130.1.1.255", "mac_address": "5c:b6:26:c0:86:61", "speed": 100, "mtu": 1500, "wake_on_lan": false, "dns_servers": [], "enabled": false}}, "file_system": {"uuid": "8b533e31-04e9-4838-839d-0656ace3e57a", "folders": {"b450c223-872c-4fe0-90cc-9da80973eaad": {"uuid": "b450c223-872c-4fe0-90cc-9da80973eaad", "name": "downloads", "size": 1000.0, "files": {"8160e685-a76f-4171-8a12-3d6b32a9ea16": {"uuid": "8160e685-a76f-4171-8a12-3d6b32a9ea16", "name": "firefox_installer.zip", "size": 1000.0, "file_type": "ZIP"}}, "is_quarantined": false}}}, "applications": {"c82f1064-f35e-466b